In [1]:
import os
import pandas as pd

In [ ]:
df=pd.DataFrame()
for folder in os.listdir('results'):
    try:
        df=pd.concat([df,pd.read_csv(r'results/'+folder+'/results_kpi.csv')])
    except:
        print('No `results_kpi.csv` in ' + folder)
df.sort_values('p_pv')
#df.to_csv('results.csv')

In [5]:
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
import numpy as np
import scipy.optimize as optimize
df=pd.read_csv('results.csv')
df=df.loc[df['p_pv']>0.1,:]
df=df.loc[df['e_bat']!=0.1,:]

In [6]:
colorscale=[
        [0, "rgb(183,0,0)"],
        [0.1, "rgb(183,0,0)"],
        [0.1, "rgb(255,56,0)"],
        [0.2, "rgb(255,56,0)"],
        [0.2, "rgb(255,90,0)"],
        [0.3, "rgb(255,90,0)"],
        [0.3, "rgb(255,146,0)"],
        [0.4, "rgb(255,146,0)"],
        [0.4, "rgb(255,203,0)"],
        [0.5, "rgb(255,203,0)"],
        [0.5, "rgb(195,255,0)"],
        [0.6, "rgb(195,255,0)"],
        [0.6, "rgb(153,204,51)"],
        [0.7, "rgb(153,204,51)"],
        [0.7, "rgb(67,166,12)"],
        [0.8, "rgb(67,166,12)"],
        [0.8, "rgb(51,153,0)"],
        [0.9, "rgb(51,153,0)"],
        [0.9, "rgb(38,98,54)"],
        [1, "rgb(38,98,54)"],]
colorbar=dict(tickmode='array',tickvals=[0.05, 0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95],ticktext=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-90','90-100'],len=1.05,ticklabeloverflow='allow')
fig= make_subplots(1,2)
fig.add_trace(go.Heatmap(x=df['p_pv'].values,
                                y=df['e_bat'].values,
                                z=df['autarkiegrad'].values,
                                text=df['autarkiegrad'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis'
                                
),1,1)
fig.add_trace(go.Heatmap(x=df['p_pv'].values,
                                y=df['e_bat'].values,
                                z=df['eigenverbrauch'].values,
                                text=df['eigenverbrauch'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis',
                                opacity=1,                                                       
),1,2)
fig.update_layout(title='Autarkiegrad und Eigenverbrauch',width=900.5,height=500,coloraxis = {'cmin':0,'cmax':1.0,'colorscale':colorscale,'colorbar':colorbar,'autocolorscale':False},)
fig.update_yaxes(tick0=0,dtick=0.25,tickangle=270,title='C_eff_sp in kWh/MWh')
fig.update_xaxes(tick0=0,dtick=0.25,title='p_pv_sp in kWp/MWh')
fig.show()


In [7]:
df=pd.read_csv('results.csv')
df=df.loc[df['p_pv']>0.1,:]
df.loc[(df['eigenverbrauch']>0)==0,'eigenverbrauch']=1
#df=df.loc[df['e_bat']!=0.1,:]
def func_autarkie(data, k1, k2,k3,k4):
    return np.maximum((k1*data[0]+k2)*(k4*data[1]+1)/((data[0]+k3)*(data[1]+1)),0)

def func_eigenverbrauch(data, k5, k6,k7,k8):
    return np.minimum((k5*data[0]+k6)*(k8*data[1]+1)/((data[0]+k7)*(data[1]+1)),1)
    
guess1 = (0.4276,-0.007667,0.4736,2.81)
params_autarkie, pcov = optimize.curve_fit(func_autarkie, [df['e_bat'].values,df['p_pv'].values], df['autarkiegrad'].values, guess1)
guess2 = (-0.001034,0.4495,0.4518,2.795)
params, pcov = optimize.curve_fit(func_eigenverbrauch, [df['e_bat'].values,df['p_pv'].values], df['eigenverbrauch'].values, guess2)

df1=pd.DataFrame()
e_bat=[]
p_pv=[]
eigen=[]
aut=[]
for bat in df['e_bat'].unique():
    for pv in df['p_pv'].unique():
        e_bat.append(bat)
        p_pv.append(pv)
        aut.append(func_autarkie([bat,pv],params_autarkie[0],params_autarkie[1],params_autarkie[2],params_autarkie[3]))#,guess1[0],guess1[1],guess1[2],guess1[3]))#
        eigen.append(func_eigenverbrauch([bat,pv],params[0],params[1],params[2],params[3]))#guess2[0],guess2[1],guess2[2],guess2[3]))
df1['e_bat']=e_bat
df1['p_pv']=p_pv
df1['eigenverbrauch']=eigen
df1['autarkiegrad']=aut
df1=df1.loc[df1['e_bat']!=0.1,:]
fig= make_subplots(1,2)
fig.add_trace(go.Heatmap(x=df1['p_pv'].values,
                                y=df1['e_bat'].values,
                                z=df1['autarkiegrad'].values,
                                text=df1['autarkiegrad'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis'
                                
),1,1)

fig.add_trace(go.Heatmap(x=df1['p_pv'].values,
                                y=df1['e_bat'].values,
                                z=df1['eigenverbrauch'].values,
                                text=df1['eigenverbrauch'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis',
                                opacity=1,                                                       
),1,2)
fig.update_layout(title='Autarkiegrad und Eigenverbrauch',width=900.5,height=500,coloraxis = {'cmin':0,'cmax':1.0,'colorscale':colorscale,'colorbar':colorbar,'autocolorscale':False},)
fig.update_yaxes(tick0=0,dtick=0.25,tickangle=270,title='C_eff_sp in kWh/MWh')
fig.update_xaxes(tick0=0,dtick=0.25,title='p_pv_sp in kWp/MWh')
fig.show()
print('Parameter Autarkie: ', params_autarkie)
print('Parameter Eigenverbrauch: ', params)

Parameter Autarkie:  [0.14333282 0.06665553 1.1357003  9.33690981]
Parameter Eigenverbrauch:  [ 2.33150631  1.29231348  1.49565706 -0.10688561]
